In [2]:
!pip install splitfolders

ERROR: Could not find a version that satisfies the requirement splitfolders (from versions: none)
ERROR: No matching distribution found for splitfolders


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
# torch imports
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models
from torch.utils.data import Dataset
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
# import splitfolders 
# import cv2
import matplotlib
from matplotlib import pyplot as plt
import math 
import time 
import os
import copy
import random
import shutil

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [72]:
class UTKDataset(Dataset):
    '''
        Inputs:
            dataFrame : Pandas dataFrame
            transform : The transform to apply to the dataset
    '''
    def __init__(self, dataFrame, transform=None):
        self.transform = transform
        
        data_holder = dataFrame.pixels.apply(lambda x: np.array(x.split(" "),dtype=float))
        arr = np.stack(data_holder)
#         arr = arr / 255.0
        arr = arr.astype('float32')
        arr = arr.reshape(arr.shape[0], 48, 48, 1)
        # reshape into 48x48x1
        self.data = arr
        
        self.age_label = np.array(dataFrame.age[:])      
        # self.gender_label = np.array(dataFrame.gender[:])
#         self.eth_label = np.array(dataFrame.ethnicity[:])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        data = self.data[index]
        data = self.transform(data)
        
        # labels = torch.tensor((self.age_label[index], self.gender_label[index]))
        # labels = torch.tensor((self.gender_label[index]))
        labels = torch.tensor((self.age_label[index]))
   
        return data, labels

In [73]:
# Hyper-parameters 
num_epochs = 30
batch_size = 32
learning_rate = 0.00001

In [74]:
dataFrame = pd.read_csv(r"C:\Users\shimi\Desktop\xuexi\413\final\age_gender.gz", compression='gzip')
# Construct age bins
# age_bins = [0,10,15,20,25,30,40,50,60,120]
# age_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8]
# dataFrame['bins'] = pd.cut(dataFrame.age, bins=age_bins, labels=age_labels)

# Split into training and testing
train_dataFrame, validation_test_dataFrame = train_test_split(dataFrame, test_size=0.2)
validation_dataFrame, test_dataFrame = train_test_split(validation_test_dataFrame, test_size=0.5)

# get the number of unique classes for each group
class_nums = {'age_num':len(dataFrame['age'].unique()), 'gen_num':len(dataFrame['gender'].unique())}

# Define train and test transforms
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.49,), (0.23,))
])

validation_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.49,), (0.23,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.49,), (0.23,))
])

# Construct the custom pytorch datasets
train_set = UTKDataset(train_dataFrame, transform=train_transform)
validation_set = UTKDataset(test_dataFrame, transform=validation_transform)
test_set = UTKDataset(test_dataFrame, transform=test_transform)

# Load the datasets into dataloaders
trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_set, batch_size=batch_size, shuffle=False)
testloader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

# Sanity Check
for data, lables in trainloader:
    print(f'Shape of training data: {data.shape}')
    print(f'Shape of lables: {lables.shape}')
    break



Shape of training data: torch.Size([32, 1, 48, 48])
Shape of lables: torch.Size([32])


In [77]:
dataFrame[10000:10500]

,Unnamed: 0,age,ethnicity,gender,img_name,pixels
10000,10000,29,1,0,20170117104547721.jpg.chip.jpg,38 72 82 77 80 79 50 27 29 34 33 44 31 34 34 1...
10001,10001,29,1,0,20170117104603706.jpg.chip.jpg,97 103 110 107 100 107 116 125 144 155 154 176...
10002,10002,29,1,0,20170117104604974.jpg.chip.jpg,119 119 121 114 49 30 34 31 33 35 39 41 42 48 ...
10003,10003,29,1,0,20170117104606138.jpg.chip.jpg,252 251 173 124 98 90 85 84 83 82 82 80 80 83 ...
10004,10004,29,1,0,20170117104625949.jpg.chip.jpg,190 190 190 188 189 184 115 80 106 140 141 146...
...,...,...,...,...,...,...
10495,10495,29,0,1,20170109134017138.jpg.chip.jpg,89 29 9 3 1 1 6 3 4 6 9 21 44 81 124 159 170 1...
10496,10496,29,0,1,20170116175630702.jpg.chip.jpg,233 241 243 247 243 237 237 240 241 239 230 21...
10497,10497,29,0,1,20170116211713681.jpg.chip.jpg,2 3 3 4 7 21 21 24 58 180 220 221 226 232 238 ...
10498,10498,29,0,1,20170117130627761.jpg.chip.jpg,22 25 30 34 51 81 89 95 98 105 113 117 118 123...


In [ ]:
class BASICCNN(nn.Module):
    def __init__(self):
        super(BASICCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 3, padding = 1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding = 1)
        self.conv3 = nn.Conv2d(128, 128, 3, padding = 1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding = 1)
        self.conv5 = nn.Conv2d(256, 256, 3, padding = 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 6 * 6, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.drop1=nn.Dropout(0.2)
        self.drop2=nn.Dropout(0.4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.drop1(x)
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = x.view(-1, 256 * 6 * 6)            
        x = F.relu(self.fc1(x)) 
        x = self.drop2(x)
        x = F.relu(self.fc2(x)) 
        x = F.relu(self.fc3(x)) 
        x = F.relu(self.fc4(x)) 
        
        return x

In [ ]:
model = BASICCNN().to(device)

criterion = nn.MSELoss()
# criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.6)

n_total_steps = len(trainloader)

lowest_loss = []
# accuracy_list = []
for epoch in range(num_epochs):
    lowest_in_epoch = math.inf
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(trainloader):

        images = images.to(device)
        labels = labels.to(device)
        labels = labels.float()
        print(labels)
        # Forward pass
        outputs = model(images).flatten()
        print(outputs)
        # print(f'Shape of training data: {outputs.shape}')
        # print(f'Shape of lables: {labels.shape}')
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
        # _, pred = torch.max(outputs.data, 1)
        # total += labels.size(0)
        # correct += (pred == labels).sum()

        if loss.item() < lowest_in_epoch:
            lowest_in_epoch = loss.item()
    lowest_loss.append((epoch, lowest_in_epoch))
    # accuracy = 100 * correct / total
    # accuracy_list.append((epoch, accuracy.item()))
    scheduler.step()
print(lowest_loss)
# print(accuracy_list)

In [ ]:
print(accuracy_list)
print(lowest_loss)

In [ ]:
correct = 0
total = 0
# Iterate through test dataset
for images, labels in testloader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, pred = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (pred == labels).sum()

accuracy = 100 * correct / total
print(accuracy.item())

In [ ]:
x_list = []
y_list = []
for i in range(num_epochs):
    x_list.append(i)
    y_list.append(lowest_loss[i][1])
plt.plot(x_list, y_list)
plt.xlabel("epoch")
plt.ylabel("Lowest Loss")
plt.show